<a href="https://colab.research.google.com/github/spozi/gpu-svgpm/blob/main/GPFeatureGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U deap imbalanced-learn scikit-learn-intelex

In [2]:
import sys
import os
import site
sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
#@title SVGPM Configuration
#@markdown ---
#@markdown ### Enter a file path:
train_file_path = "/content/Earthquakes_train.csv" #@param {type:"string"}
test_file_path = "/content/Earthquakes_test.csv" #@param {type:"string"}

#@markdown ---
#@markdown ### Enter SVGPM Parameters:
population_size = 500 #@param {type:"slider", min:0, max:1000, step:2}
number_of_generation = 250 #@param {type:"slider", min:0, max:1000, step:2}

#@markdown ### Enter 2^C and 2^Gamma Parameter:
C = -2 #@param {type:"slider", min:-10, max:10, step:1}
gamma = -3 #@param {type:"slider", min:-10, max:10, step:1}
#@markdown ---

In [4]:
from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#Train and test file
train_file = train_file_path
test_file = test_file_path

train_data = np.loadtxt(train_file, delimiter=",", skiprows=1)
test_data = np.loadtxt(test_file, delimiter=",", skiprows=1)

X_train = train_data[:, 1:]
X_test = test_data[:, 1:]

#Scale each feature to 0 and 1
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#y_train
y_train = train_data[:, 0].astype(int)
y_test = test_data[:, 0].astype(int)

#Compute class weight
unique, counts = np.unique(y_train, return_counts=True)
pos_index = 0
neg_index = 0
if counts[0] < counts[1]:
  pos_index = 0
  neg_index = 1
else:
  pos_index = 1
  neg_index = 0

class_weight = {pos_index: counts[neg_index], neg_index: counts[pos_index]}
# if pos_index == 0:
#   class_weight = {pos_index: counts[neg_index]} #Total of negative class: Total of positive class
# elif pos_index == 1:
#   class_weight = {pos_index: counts[neg_index]}

Version 2 GP Feature Generator

In [5]:
import random
import operator
import math
import statistics

import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

nFeatures = X_train.data.shape[1]
pset = gp.PrimitiveSet("MAIN", nFeatures) 
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.erfc, 1)
pset.addPrimitive(math.erf, 1)
pset.addPrimitive(math.exp, 1)
pset.addPrimitive(math.gamma, 1)
pset.addPrimitive(math.sqrt, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand1", lambda: round(random.uniform(0.1, 1.0), 10))

creator.create("FitnessMax", base.Fitness, weights=(1.0, 0.8))
creator.create("Tree", gp.PrimitiveTree)
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("main_expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=5)
toolbox.register('MAIN', tools.initIterate, creator.Tree, toolbox.main_expr)

func_cycle = [toolbox.MAIN]

toolbox.register("individual", tools.initCycle, creator.Individual, func_cycle)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from multiprocessing import Pool

from time import time


import warnings
warnings.filterwarnings("ignore")

def evalSymbReg(score):
  return (score[0], score[1])

def evalSymbRegPop(population):
  # Evaluate each individual in population
  #1. Compute the expression of every individual
  list_vecs = []
  # start = time()
  for individual in population:
    #The following code should be optimized/vectorized
    #Evaluating expression on each vector
    func = toolbox.compile(expr=individual)
    vec = []
    for x in X_train: #Iterate every vector x (row) in data (matrix) X
      try:
        val = func(*x)
        vec.append(val)
      except:
        vec.append(0)
    list_vecs.append(vec)
  end = time()
  # expression_evaluation_time = end - start

  #2. Convert list_vecs to numpy array
  evaluated_X = np.array(list_vecs).T
  evaluated_X = np.float32(evaluated_X)
  evaluated_X = np.nan_to_num(evaluated_X, copy=True, nan=0.0, posinf=0.0, neginf=0.0)
  evaluated_X_train = np.hstack((X_train, evaluated_X)) #Merge x_train with evaluated_x

  #3. Individual (feature) selection
  # https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel

  # start = time()
  clf = ExtraTreesClassifier(n_estimators=50)
  clf = clf.fit(evaluated_X_train, y_train)

  #4. Extract features that at top threshold (get the 50 percentile)
  q1 = np.percentile(clf.feature_importances_, 50)                                        #Get the top 50 percentile features
  features = [True if val >= q1 else False for val in clf.feature_importances_.tolist()]  #Get the features indices
  features = features[X_train.shape[1]:]                                                  #Extract GP Features from all features
  X_train_new = evaluated_X[:, features]                                                  #Form a new training data with GP Features
  # end = time()
  # feature_evaluation_time = end - start

  #5. Merge X_train with X_train_new row-wise
  X_train_new = np.hstack((X_train, X_train_new))                                         #Merge back original training data with new GP based training data

  #6. Use svc to get total nSV
  # start = time()
  clf_svc = SVC(C=2**C, gamma=2**gamma, class_weight=class_weight)      #Based on original paper 
  # clf_svc = SVC(C=1, kernel='linear')                                 #Smaller C value will increase the margin size of the hyperplane. The new features should be generate such that it has very high linear seperability
  clf_svc.fit(X_train_new, y_train)                                     #98% of runtime is at here
  # end = time()
  # svm_training_time = end - start

  # print("Expression evaluation: %.4f, Feature evaluation: %.4f, SVM training: %.4f" % (expression_evaluation_time, feature_evaluation_time, svm_training_time))

  y_pred = clf_svc.predict(X_train_new)

  #7. Compute the score
  nSV = clf_svc.support_vectors_.shape[0]
  tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()
  f1 = f1_score(y_train, y_pred)
  accuracy = accuracy_score(y_train, y_pred)
  specificity = tp / (tp + fn)
  sensititivy = tn / (tn + fp)
  gmean = math.sqrt(specificity * sensititivy)
  # print(gmean,accuracy,min(gmean,accuracy)/nSV)
  fitness = (min(gmean,accuracy), min(gmean,accuracy)/nSV)

  #8. Output the fitness value
  ind_pop_fitness = []
  for f in features:
    if f is True:
      ind_pop_fitness.append(fitness) #If the feature is in the tree, set the fitness value to fitness
    else:
      ind_pop_fitness.append((0,0))     #If the feature is in the tree, set the fitness value to 0
  
  return ind_pop_fitness

psets = [pset]
toolbox.register("compile", gp.compileADF, psets=psets)
toolbox.register('evaluate', evalSymbReg)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register("expr", gp.genFull, min_=1, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=80))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=80))

In [7]:
def main():
  random.seed(1024)
  ind = toolbox.individual()
  
  pop = toolbox.population(n=population_size)
  hof = tools.HallOfFame(population_size)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)

  logbook = tools.Logbook()
  logbook.header = "gen", "evals", "std", "min", "avg", "max"

  CXPB, MUTPB, NGEN = 0.5, 0.2, number_of_generation

  # # Evaluate the entire population
  #1. Compute the metric on the set of individuals
  ind_pop_fitness = evalSymbRegPop(pop)

  #2. Then, determine the best individual using toolbox
  for ind, fitness in zip(pop, ind_pop_fitness):
    ind.fitness.values = toolbox.evaluate(fitness)

  hof.update(pop)
  record = stats.compile(pop)
  logbook.record(gen=0, evals=len(pop), **record)
  print(logbook.stream)

  for g in range(1, NGEN):
    # Select the offspring
    offspring = toolbox.select(pop, len(pop))
    # Clone the offspring
    offspring = [toolbox.clone(ind) for ind in offspring]

    # Apply crossover and mutation
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        for tree1, tree2 in zip(ind1, ind2):
            if random.random() < CXPB:
                toolbox.mate(tree1, tree2)
                del ind1.fitness.values
                del ind2.fitness.values

    for ind in offspring:
        for tree, pset in zip(ind, psets):
            if random.random() < MUTPB:
                toolbox.mutate(individual=tree, pset=pset)
                del ind.fitness.values
                        
    # Evaluate the individuals with an invalid fitness
    invalids = [ind for ind in offspring if not ind.fitness.valid]

    #1. Compute the metric on the set of individuals
    ind_pop_invalid_fitness = evalSymbRegPop(invalids)

    #2. Then, determine the best individual using toolbox
    for ind, fitness in zip(invalids, ind_pop_invalid_fitness):
      ind.fitness.values = toolbox.evaluate(fitness)
            
    # Replacement of the population by the offspring
    pop = offspring
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, evals=len(invalids), **record)
    print(logbook.stream)
  
  print('Best individual : ', hof[0][0], hof[0].fitness)
  return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()

0.9737945687202026 0.9906832298136646 0.0030526475508470302
gen	evals	std     	min	avg     	max     
0  	500  	0.418392	0  	0.239328	0.973795
0.9866533578135983 0.9782608695652174 0.003038077234674588
1  	286  	0.46049 	0  	0.329852	0.978261
1.0 1.0 0.003105590062111801
2  	301  	0.468126	0  	0.340565	1       
0.9559252603882146 0.984472049689441 0.002977960312735871
3  	307  	0.464338	0  	0.340741	1       
0.982607368881035 0.9937888198757764 0.003080273883639608
4  	281  	0.472854	0  	0.35942 	1       
0.982607368881035 0.9937888198757764 0.003099707788268249
5  	272  	0.474377	0  	0.359053	1       
0.9737945687202026 0.9906832298136646 0.0038188022302753045
6  	306  	0.461608	0  	0.323384	1       
1.0 1.0 0.003105590062111801
7  	291  	0.467121	0  	0.329379	1       
0.9913418283769 0.9968944099378882 0.0030787013303630434
8  	308  	0.473816	0  	0.350126	1       
1.0 1.0 0.003105590062111801
9  	322  	0.470367	0  	0.335409	1       
0.9913418283769 0.9968944099378882 0.003088292300239

**Prediction task**

1.   Create new data from the list of fittest individual (fittest features) for both training and testing data.
2.   Fit the svm with transformed training data
3.   Predict the transformed testing data



In [10]:
#1. Evaluate the expression for training set
list_train_vecs = []
max_value = max([individual.fitness.values[0] for individual in hof])

for individual in hof:
  if individual.fitness.values[0] == max_value:
      func = toolbox.compile(expr=individual)
      vec = []
      for x_train in X_train: #Iterate every vector x (row) in data (matrix) X
        try:
          val = func(*x_train)
          vec.append(val)
        except:
          vec.append(0)
      list_train_vecs.append(vec)

#2. Evaluate the expression for testing set
list_test_vecs = []
for individual in hof:
  if individual.fitness.values[0] == max_value:
      func = toolbox.compile(expr=individual)
      vec = []
      for x_test in X_test: #Iterate every vector x (row) in data (matrix) X
        try:
          val = func(*x_test)
          vec.append(val)
        except:
          vec.append(0)
      list_test_vecs.append(vec)

#3. Convert list_vecs to numpy array
X_train_new = np.array(list_train_vecs).T   #Need to refactor X_train g
X_train_new = np.nan_to_num(X_train_new, copy=True, nan=0.0, posinf=0.0, neginf=0.0)
X_train_new = np.hstack((X_train, X_train_new))

X_test_new = np.array(list_test_vecs).T   #Need to refactor X_test g
X_test_new = np.nan_to_num(X_test_new, copy=True, nan=0.0, posinf=0.0, neginf=0.0)
X_test_new = np.hstack((X_test, X_test_new))

SVGPM V2 - Accuracy-Score:  0.7482014388489209
SVGPM V2 - F1-Score:  0.0


In [11]:
#Additional evaluation
#1. Fit the SVM
list_accuracy_result = []
list_f1_result = []
for c in range(-5,6):
  for g in range(-4,6):
    clf_svc = SVC(C=2**c, gamma=2**g, class_weight=class_weight)
    clf_svc.fit(X_train, y_train)
    y_pred = clf_svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    list_accuracy_result.append(accuracy)
    f1 = f1_score(y_test, y_pred)
    list_f1_result.append(f1)

print("SVM - Highest Accuracy-Score: ", max(list_accuracy_result))
print("SVM - Highest F1-Score: ", max(list_f1_result))


#2. Fit the SVM (SVGPM)
list_accuracy_result = []
list_f1_result = []
for c in range(-5,6):
  for g in range(-4,6):
    clf_svc = SVC(C=2**c, gamma=2**g, class_weight=class_weight)
    clf_svc.fit(X_train_new, y_train)
    y_pred = clf_svc.predict(X_test_new)
    accuracy = accuracy_score(y_test, y_pred)
    list_accuracy_result.append(accuracy)
    f1 = f1_score(y_test, y_pred)
    list_f1_result.append(f1)

print("SVGPM V2 - Highest Accuracy-Score: ", max(list_accuracy_result))
print("SVGPM V2 - Highest F1-Score: ", max(list_f1_result))

SVM - Highest Accuracy-Score:  0.7482014388489209
SVM - Highest F1-Score:  0.0
SVGPM V2 - Highest Accuracy-Score:  0.7482014388489209
SVGPM V2 - Highest F1-Score:  0.40229885057471265
